<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_04_1_langchain_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 7: LangChain: Agents**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 7 Material

* **Part 7.1: Structured Output Parser** [[Video]]() [[Notebook]](t81_559_class_07_1_agents.ipynb)
* Part 7.2: Other Parsers (CSV, JSON, Pandas, Datetime)** [[Video]]() [[Notebook]](t81_559_class_05_2_parsers.ipynb)
* Part 7.3: Pydantic Parser [[Video]]() [[Notebook]](t81_559_class_05_3_pydantic.ipynb)
* Part 7.4: Custom Output Parser [[Video]]() [[Notebook]](t81_559_class_05_4_custom_parsers.ipynb)
* Part 7.5: Output-Fixing Parser [[Video]]() [[Notebook]](t81_559_class_05_5_output_fixing_parsers.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [2]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai,langchain_experimental,duckduckgo-search

Note: using Google CoLab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
  Att

# 7.1: LangChain Agents



The following code uses an LLM to tell a joke. The pydantic parser ensures that the joke ends in a question mark. Such checking ensures that the somewhat random LLM produces output that aligns with our expectations.

In [5]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()

python_repl.run("print(1+1)")

'2\n'

In [13]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
search.run("current version of OSX")

"The first ancestor of the macOS family was Mac OS X Public Beta released in 2000, followed by a public release of Mac OS X 10.0 in 2001. Let's recount the stories of all Mac OS X versions, up to the current macOS (or check out our ultimate infographic.) 1. Mac OS X 10.0 (Cheetah) March 24, 2001: Aqua interface is born with Mac OS X Cheetah ... macOS 13 Ventura. Introduced at WWDC 2022, macOS Ventura is the current version of macOS, the operating system that runs on the Mac. macOS Ventura is a significant update that introduces a new ... macOS 14 Sonoma: Release date. Apple macOS 14 Sonoma was released on September 26 as a free update for all supported Macs. (Image credit: Apple) Of course, any new MacBook or desktop Mac you buy ... The newest version of macOS, aka macOS 14, aka Sonoma (named after the Californian wine region), is now available. Among the new features Sonoma brings to Macs are changes to widgets, a new Game ... Current Version. The current version of macOS Sonoma is ma

In [8]:
!pip install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 33.0 MB/s eta 0:00:00


In [20]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(a: int, b: int) -> int - Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [21]:
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI

MODEL = 'gpt-3.5-turbo'
TEMPERATURE = 0.2

llm = ChatOpenAI(
  model=MODEL,
  temperature=TEMPERATURE,
  max_tokens= 256)

tools = [multiply]
agent_chain = initialize_agent(tools,
                               llm,
                               agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True)


agent_chain("What is 2 multiplied by 3?")



> Entering new AgentExecutor chain...
Thought: I can use the multiply tool to find the product of 2 and 3. 

Action:
```
{
  "action": "multiply",
  "action_input": {"a": 2, "b": 3}
}
```  

Observation: 6
Thought:I have calculated that 2 multiplied by 3 equals 6.

> Finished chain.


{'input': 'What is 2 multiplied by 3?',
 'output': 'I have calculated that 2 multiplied by 3 equals 6.'}

https://python.langchain.com/docs/integrations/chat/